## Lecture 3: Convex Optimization

In [3]:
%matplotlib inline
from IPython.core.display import HTML, Image
from IPython.display import YouTubeVideo
from sympy import init_printing, Matrix, symbols, Rational
import sympy as sym
from warnings import filterwarnings
init_printing(use_latex = 'mathjax')

import numpy as np

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
              'theme': 'simple',
              'start_slideshow_at': 'selected',
})
import warnings
warnings.simplefilter("ignore")

## Outline for this Lecture
- Convexity
    - Convex Set
    - Convex Function
- Introduction to Optimization
- Introduction to Lagrange Duality
* Unconstrained Optimization
* Constrained Optimization
    * Langrange duality
    * KKT conditions
    * Convex function

> In this lecture, we will first introduce convex set, convex function and optimization problem. One approach to solve optimization problem is to solve its dual problem. We will briefly cover some basics of duality in this lecture. More about optimization and duality will come when we study support vector machine (SVM).

## Convexity

### Convex Sets
- $C \subseteq \mathbb{R}^n$ is **convex** if
    $$t x + (1-t)y \in C$$
    for any $x, y \in C$ and $0 \leq t \leq 1$
- that is, a set is convex if the line connecting **any** two points in the set is entirely inside the set
<center>
<div class="image" style="width:450px">
  <img src="images/Convex_Set.png">
</div>
</center>
<center><span>(Left: Convex Set; Right: Non-convex Set)</span></center>


### Convex Functions
* We say that a function $f$ is *convex* if, for any distinct pair of points $x,y$ we have
    $$
    f(tx_1+(1-t)x_2) \leq tf(x_1)+(1-t)f(x_2) \quad \forall t \in[0,1]
    $$
* A function $f$ is said to be *concave* if $-f$ is convex
<center>
<div class="image" style="width:500px">
  <img src="images/Convex_Fun.png">
</div>
</center>

### Fun Facts About Convex Functions
* If $f$ is differentiable, then $f$ is convex iff $f$ "lies above its linear approximation", i.e.:
    $$
    f(x + y) \geq f(x) + \nabla_x f(x) \cdot y \quad \forall x,y
    $$
* If $f$ is twice-differentiable, then the hessian is always positive semi-definite!
* This last one you will show on your homework :-)

### For unconstrained optimization

* **Stationary point $\Rightarrow$ global minimizer**
* **Uniqueness of global minimizer for strictly convex function** (HW1 Q5)

### For constrained optimization
* For convex problem
    * Use Slater's conditions to get strong duality
    * Use KKT conditions to get strong duality and find the solutions

## Introduction to Optimization

### The Most General Optimization Problem
- Assume $f$ is some function, and $C \subset \mathbb{R}^n$ is some set. The following is an *optimization problem*:
    $$
    \begin{array}{ll}
    \mbox{minimize} & f(x) \\
    \mbox{subject to} & x \in C
    \end{array}
    $$
-  How hard is it to find a solution that is (near-) optimal? This is one of the fundamental problems in Computer Science and Operations Research.
- A huge portion of ML relies on this task

### A Rough Optimization Hierarchy
- For optimization problem
    $$
    \mbox{minimize } \ f(x) \quad \mbox{subject to } x \in C
    $$
* **[Really Easy]** $C = \mathbb{R}^n$ (i.e. problem is *unconstrained*), $f$ is convex, $f$ is differentiable, strictly convex, and "slowly-changing" gradients
* **[Easyish]** $C = \mathbb{R}^n$, $f$ is convex
* **[Medium]** $C$ is a convex set, $f$ is convex
* **[Hard]** $C$ is a convex set, $f$ is non-convex
* **[REALLY Hard]** $C$ is an arbitrary set, $f$ is non-convex

### Optimization Without Constraints
- Optimization problem without constraint is given by
    $$
    \begin{array}{ll}
    \mbox{minimize} & f(x) \\
    \mbox{subject to} & x \in \mathbb{R}^n
    \end{array}
    $$
- This problem tends to be easier than constrained optimization
- We just need to find an $x$ such that $\nabla f(x) = \vec{0}$
- Techniques like *gradient descent* or *Newton's method* work in this setting. (More on this later)

### Optimization With Constraints
- Optimization problem with constraint is given by
    $$
    \begin{aligned}
    & {\text{minimize}} & & f(\mathbf{x})\\
    & \text{subject to} & & g_i(\mathbf{x}) \leq 0, \quad i = 1, ..., m\\
    & & & h_j(x) = 0, \quad j = 1, ..., n
    \end{aligned}
    $$
- Here $C = \{ x : g_i(x) \leq 0,\ h_j(x) = 0, \ i=1, \ldots, m,\ j = 1, ..., n \}$
- $C$ is convex as long as all $g_i(x)$ convex and all $h_j(x)$ affine
- The solution of this optimization may occur in the *interior* of $C$, in which case the optimal $x$ will have $\nabla f(x) = 0$
- But what if the solution occurs on the *boundary* of $C$?
<center>
<div class="image" style="width:500px">
  <img src="images/lagrange.gif">
</div>
</center>

## Introduction to Lagrange Duality
- In some cases original (**primal**) optimization problem can hard to solve, solving a proxy problem sometimes can be easier
- The proxy problem could be **dual** problem which is transformed from primal problem
- Here is how to transform from primal to dual. For primal problem
    $$
    \begin{aligned}
    & {\text{minimize}} & & f(\mathbf{x})\\
    & \text{subject to} & & g_i(\mathbf{x}) \leq 0, \quad i = 1, ..., m\\
    & & & h_j(x) = 0, \quad j = 1, ..., n
    \end{aligned}
    $$
    Its Lagrangian is 
    $$
    L(x,\boldsymbol{\lambda}, \boldsymbol{\nu}) := f(x) + \sum_{i=1}^m \lambda_i g_i(x) + \sum_{j=1}^n \nu_j h_j(x)
    $$
    of which $\boldsymbol{\lambda} \in \mathbb{R}^m$, $\boldsymbol{\nu} \in \mathbb{R}^n$ are **dual variables**

- The **Langrangian dual function** is 
    $$
    L_D(\boldsymbol{\lambda}, \boldsymbol{\nu}) \triangleq \underset{x}{\inf}L(x,\boldsymbol{\lambda}, \boldsymbol{\nu}) = \underset{x}{\inf} \ \left[ f(x) + \sum_{i=1}^m \lambda_i g_i(x) + \sum_{j=1}^n \nu_j h_j(x) \right] 
    $$
- The minimization is usually done by finding the stable point of $L(x,\boldsymbol{\lambda}, \boldsymbol{\nu})$ with respect to $x$

### The Lagrange Dual Problem

* Then the **dual problem** is 
    $$
    \begin{aligned}
    & {\text{maximize}} & & L_D(\mathbf{\lambda}, \mathbf{\nu})\\
    & \text{subject to} & & \lambda_i, \nu_j \geq 0 \quad i = 1, \ldots, m ,\ j = 1, ..., n\\
    \end{aligned}
    $$
- Instead of solving primal problem with respect to $x$, we now need to solve dual problem with respect to $\mathbf{\lambda}$ and $\mathbf{\nu}$
    * $ L_D(\mathbf{\lambda}, \mathbf{\nu})$ is concave even if primal problem is not convex
    * Let the $p^*$ and $d^*$ denote the optimal values of primal problem and dual problem, we always have *weak duality*: $p^* \geq d^*$
    * Under nice conditions, we get *strong duality*: $p^* = d^*$
    * Many details are omitted here and they will come when we study **support vector machine (SVM)**

## Unconstrained Optimization

* ** Differentiable **: $\forall x$, the gradient $\nabla f(x)$ exists
* ** Twice differentiable **: $\forall x$, the Hessian matrix $\nabla^2 f(x)$ exists
* **Stationary point**: $x$ where $\nabla f(x) = \vec{0}$
* ** Saddle point **: a stationary point but not a local minimizer/maximizer
    * e.g. $x=0$ for $f(x) = x^3$


* **Problem formulation**
$$
\begin{align*}
\min_{x\in \mathbb{R}^d} \quad& f(x)
\end{align*}
$$


* We need to find the stationary points
* Global minimizer $\Rightarrow$ local minimizer $\Rightarrow$ stationary point
* Solving the equation: closed-form solutions, gradient descent, Newton's method, etc.
**Stationary point $\Rightarrow$ local minimizer $\Rightarrow$ global minimizer?**

* Finite # of stationary points: check everyone
* Differentiable convex function: global minimizer! $$f(y)\geq f(x)+\langle\nabla f(x),y-x\rangle=f(x)$$
* Twice continuous differentiable function: 
    $$
    \begin{align*}
    f(y) &= f(x) + \langle\nabla f(x), y-x\rangle + \frac{1}{2} \langle y-x, \nabla^2 f(x)(y-x) \rangle \\&+ o(\Vert y-x\Vert^2)\end{align*}$$
    * $f(x+tv) - f(x) = t^2\left(\frac{1}{2}\langle v, \nabla^2 f(x)v \rangle + \frac{o(t^2)}{t^2}\right)$ where we let $\Vert v\Vert=1$ ($v$ on the unit ball centered at $x$)
    * $\nabla^2 f(x)$ positive definite $\Rightarrow$ local minimizer, how about PSD?
   
* ** Property**: differentiable, local minimizer $x^*$ $\Rightarrow$ $\nabla f(x) = \vec{0}$ (*the inverse is not true*)
* ** Property**: twice differentiable, local minimizer $x^*$ $\Rightarrow$ $\nabla^2 f(x)$ is PSD (*the inverse is not true*)

## Constrained Optimization

* **Problem formulation**
$$
\begin{align*}
\min_{x\in \mathbb{R}^d} \quad& f(x) \\
\text{s.t.} \quad& g_i(x) \leq 0, \quad i = 1, \ldots, m\\
 & h_j(x) = 0, \quad j = 1, \ldots, n
\end{align*}
$$


* The set $C=\{ x\vert g_i(x) \leq 0, h_j(x) = 0, i=1, \ldots, m, j=1, \ldots, n \}$ is convex if $g_i(x)$ convex and $h_j(x)$ affine
* The solution of this optimization may occur in the *interior* of $C$, in which case the optimal $x$ will have $\nabla f(x) = 0$
* But what if the solution occurs on the *boundary* of $C$?

### Langrange Duality

$$
\begin{align*}
\min_{x\in \mathbb{R}^d} \quad& f(x) \\
\text{s.t.} \quad& g_i(x) \leq 0, \quad i = 1, \ldots, m\\
& h_j(x) = 0, \quad j = 1, \ldots, n
\end{align*}
$$
* **Lagrangian function**
$$
L(x,\boldsymbol{\lambda}, \boldsymbol{\nu}) := f(x) + \sum_{i=1}^m \lambda_i g_i(x) + \sum_{j=1}^n \nu_j h_j(x)
$$
* **Lagrange multipliers/dual variables**: $\boldsymbol{\lambda} \in \mathbb{R}^m$ and $\boldsymbol{\nu} \in \mathbb{R}^n$ 

* $$
L(x,\boldsymbol{\lambda}, \boldsymbol{\nu}) := f(x) + \sum_{i=1}^m \lambda_i g_i(x) + \sum_{j=1}^n \nu_j h_j(x)
$$


* **Primal function**
$$
\begin{align*}
L_{P}(x)=
\left\{\begin{array}{r} \max_{\boldsymbol{\lambda}, \boldsymbol{\nu}}\, L(x,\boldsymbol{\lambda}, \boldsymbol{\nu}) \\ \text{s.t.}\quad \lambda_i\geq 0\end{array}\right.
=\left\{\begin{array}{ll}f(x) & x\in C\\ +\infty & x\notin C \end{array}\right.
\end{align*}
$$
where $ C = \{ x\vert g_i(x) \leq 0, h_j(x) = 0, i=1, \ldots, m, j=1, \ldots, n \}$


* What's the difference between $f(x)$ and $L_P(x)$?

Hence the original optimization is equivalent to the **primal problem**:
$$\min_{x\in \mathbb{R}^n} L_P(x)$$
or
$$
\begin{align*}
\min_{x\in \mathbb{R}^d} \max_{\boldsymbol{\lambda}\in \mathbb{R}^m, \boldsymbol{\nu}\in \mathbb{R}^n}\, &L(x,\boldsymbol{\lambda}, \boldsymbol{\nu})\\
\text{s.t.}\quad &\lambda_i \geq 0
\end{align*}
$$


* It is actually a double optimization (inner optimization in exchange of unconstrained $x$)

* Swap the outer and inner optimization to get the **dual problem**:
$$
\begin{align*}
\max_{\boldsymbol{\lambda}\in \mathbb{R}^m, \boldsymbol{\nu}\in \mathbb{R}^n}\min_{x\in \mathbb{R}^d}\, &L(x,\boldsymbol{\lambda}, \boldsymbol{\nu})\\
\text{s.t.}\quad &\lambda_i \geq 0
\end{align*}
$$

** Dual function **
$$L_D(\boldsymbol{\lambda}\in \mathbb{R}^m, \boldsymbol{\nu}\in \mathbb{R}^n)=\min_{x\in \mathbb{R}^d}\, L(x,\boldsymbol{\lambda}, \boldsymbol{\nu})$$

What's the connection between primal and dual problem?
* Primal solution: $x^*$, dual solution: $\lambda^*$ and $\nu^*$

* **Weak duality** (always true)
$$d^*=L_D(\boldsymbol{\lambda}^*, \boldsymbol{\nu}^*)\leq p^*=L_P(x^*)=f(x^*)$$

* **Strong duality** (under additional conditions): $d^* = p^* =f(x^*)= L(x^*,\boldsymbol{\lambda}^*, \boldsymbol{\nu}^*)$

$$
\begin{align*}
L(x^*,\boldsymbol{\lambda}^*, \boldsymbol{\nu}^*)
\leq p^*=\max_{\boldsymbol{\lambda}, \boldsymbol{\nu}}\, L(x^*,\boldsymbol{\lambda}, \boldsymbol{\nu})
\leq d^*=\min_x\, L(x,\boldsymbol{\lambda}^*, \boldsymbol{\nu}^*)\leq L(x^*,\boldsymbol{\lambda}^*,\boldsymbol{\nu}^*)
\end{align*}
$$
* Sometimes the dual problem is easier. But when strong duality holds?

### Karush–Kuhn–Tucker (KKT) conditions
* Suppose differentiable $f(x),g_i(x),h_j(x)$
* (Primal feasibility) $$g_i(x)\leq 0, h_j(x)=0$$
* (Dual feasibility) $$\lambda_i\geq 0$$
* (Stationarity) $$\nabla_x L(x, \boldsymbol{\lambda}, \boldsymbol{\nu})=0$$
* (Complementary slackness) $$\lambda_i g_i(x)=0$$

### Necessary conditions of strong duality
* Strong duality $\Rightarrow$ KKT conditions hold for $x^*$, $\lambda^*$ and $\nu^*$ (Proof?)


### Sufficient conditions of strong duality
* KKT conditions hold for $\tilde{x}$, $\tilde{\boldsymbol{\lambda}}$ and $\tilde{\boldsymbol{\nu}}$ $\Rightarrow$ strong duality, primal solution $\tilde{x}$, dual solution $\tilde{\boldsymbol{\lambda}}$ and $\tilde{\boldsymbol{\nu}}$ (Proof?)
* For convex problem: $f(x)$ and $g_i(x)$ are convex, $h_j(x)$ are affine
* Slater's conditions: $\exists x$ s.t. $g_i(x)<0$ and $h_j(x)=0$
    


## Recommended reading:
* Free online!
* Chapter 5 covers duality
<center>
<div class="image" style="width:200px">
  <img src="images/bv_cvxbook_cover.jpg">
</div>
</center>